Nombre: David Díaz Paz y Puente | Matrícula: 650794

# A1.3 Solución de problemas y selección de características

## 1. Introducción

Para esta actividad partimos de la idea de que en ciencia de datos casi nunca recibimos conjuntos de datos “listos para usar”: es normal encontrar variables categóricas mal codificadas, escalas inconsistentes, valores atípicos y hasta información redundante entre variables, lo cual puede afectar directamente la calidad y la interpretabilidad de cualquier modelo predictivo. 

Trabajaremos con una base de datos que incluye información demográfica y académica de estudiantes, además de sus calificaciones parciales y final. El objetivo del reporte es construir un modelo de regresión lineal múltiple para predecir la calificación final, enfrentando explícitamente los retos de usar datos reales y justificando la selección de variables explicativas más relevantes. Para lograrlo, se describirá el proceso de exploración, limpieza y transformación de datos, se analizarán relaciones entre variables para detectar colinealidad o redundancia, y finalmente se aplicará un criterio de selección de características que permita quedarse con un subconjunto informativo. Con esas variables seleccionadas, se entrenará y evaluará el modelo comparando su desempeño en entrenamiento y prueba (evitando fuga de datos), y se cerrará con una reflexión crítica sobre limitaciones y posibles mejoras. 

In [1]:
import pandas as pd

ruta = f'calificaciones.csv'
df = pd.read_csv(ruta)

## 2. Metodología

### 2.1 Exploración y comprensión del conjunto de datos

De acuerdo a la fuente, la base de datos utilizada reúne información real de estudiantes de nivel secundaria en dos escuelas de Portugal, y está diseñada para relacionar el desempeño académico con características del estudiante y de su contexto. Se compone de atributos (variables) que incluyen: (1) calificaciones por periodo (G1, G2) y la calificación final del año (G3), (2) datos demográficos (por ejemplo, edad y sexo), (3) factores sociales y familiares (como apoyo en casa o condiciones del entorno), y (4) variables escolares (relacionadas con hábitos o dinámica académica).

In [3]:
print(df.head())
print(df.info())

  Escuela Sexo  Edad  HorasDeEstudio  Reprobadas Internet  Faltas  G1  G2  G3
0      GP    F    18               2           0       no       6   5   6   6
1      GP    F    17               2           0      yes       4   5   5   6
2      GP    F    15               2           3      yes      10   7   8  10
3      GP    F    15               3           0      yes       2  15  14  15
4      GP    F    16               2           0       no       4   6  10  10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Escuela         395 non-null    object
 1   Sexo            395 non-null    object
 2   Edad            395 non-null    int64 
 3   HorasDeEstudio  395 non-null    int64 
 4   Reprobadas      395 non-null    int64 
 5   Internet        395 non-null    object
 6   Faltas          395 non-null    int64 
 7   G1              395 non-nu

A partir del resumen del DataFrame (_df_), se identifica que el dataset está compuesto por 395 observaciones y 10 variables, donde cada registro representa a un estudiante con características académicas, personales y de contexto escolar. Esta primera revisión permite clasificar las variables según su naturaleza, lo cual es clave para decidir cómo tratarlas dentro de un modelo de regresión lineal. A continuación se presenta la descripción para cada variable de acuerdo a la fuente de la base de datos ([archive.ics.uci.edu](https://archive.ics.uci.edu/dataset/320/student+performance)):

| Variable Name | Role | Type | Descripction |
|-----------|-----------|-----------|-----------|
| school | Feature | Categorical | student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira) | 
| sex | Feature | Binary | student's sex (binary: 'F' - female or 'M' - male) |
| age | Feature | Integer |  student's age (numeric: from 15 to 22)  |
| studytime | Feature | Integer   | weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours) |
| failures | Feature | Integer   | number of past class failures (numeric: n if 1<=n<3, else 4) |
| internet | Feature | Binary   | wants to take higher education (binary: yes or no) |
| absences | Feature | Integer   | number of school absences (numeric: from 0 to 93) |
| G1 | Target | Categorical |   first period grade (numeric: from 0 to 20) |
| G2 | Target | Categorical |   second period grade (numeric: from 0 to 20) |
| G3 | Target | Categorical |   final grade (numeric: from 0 to 20, output target) |


Por un lado, se tienen variables cuantitativas discretas como Edad, HorasDeEstudio, Reprobadas, Faltas, G1, G2 y G3, que pueden incorporarse directamente al modelo porque ya están en formato numérico. Estas variables son candidatas naturales a explicar la calificación final, ya que miden comportamientos o antecedentes académicos que pueden tener relación directa con el rendimiento.

Por otro lado, existen variables cualitativas nominales como Escuela, Sexo e Internet, que no poseen un orden numérico. Estas no pueden introducirse directamente en una regresión lineal, por lo que requieren codificación previa (por ejemplo, variables dummy u one-hot encoding). Este proceso incrementa el número de variables y puede añadir complejidad al modelo. Además, es necesario evaluar si realmente aportan información relevante o solo ruido estadístico.

Finalmente, el hecho de que no existan valores faltantes simplifica el preprocesamiento, ya que no será necesario imputar datos ni eliminar registros, permitiendo centrar el esfuerzo en análisis de correlaciones, detección de redundancias y selección de características. 